In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("target_table", "f_inventory")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_DWH_PRD.F_STAGING_INVENTORY, VCM_STG_INF.SAP_0PLANT"
)

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory
(
  hash_id STRING,
  calday DATE,
  store_id STRING,
  product_id STRING,
  closing_stock_quantity FLOAT,
  closing_stock_value FLOAT,
  addr_numbr STRING
)""")

In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW temp_f_inventory
    AS
    WITH table_date AS (
            SELECT col AS date_gen
            FROM EXPLODE(SEQUENCE(DATE {START_DATE}, DATE {END_DATE}, INTERVAL 1 DAY)) AS date_gen
            WHERE col <= CURRENT_DATE()
    ),

    add_rec AS (
    SELECT plant, addr_numbr FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY plant ORDER BY END_DATE DESC) rn 
        FROM  {catalog_name}.udp_wcm_silver_sap_bw.sap_0plant
        WHERE IF(addr_numbr='',NULL,addr_numbr) IS NOT NULL
        )
    WHERE rn = 1
    ),

    main AS (
        SELECT * EXCEPT(rn, date_from, date_to)
        FROM ( 
            SELECT 
                date_gen calday,
                store_id,
                product_id,
                closing_stock_quantity,
                closing_stock_value,
                ROW_NUMBER() OVER (PARTITION BY store_id, product_id, CAST(closing_stock_quantity AS NUMERIC), CAST(closing_stock_value AS NUMERIC) ORDER BY date_gen ASC) rn,
                date_from,
                DATE_SUB(IFNULL(date_to, '2400-01-01'),1) date_to
            FROM table_date 
            INNER JOIN (
                SELECT
                    calday,
                    store_id,
                    product_id,
                    ROUND(SUM(closing_stock_quantity) OVER (PARTITION BY store_id, product_id ORDER BY calday ASC),3) closing_stock_quantity,
                    SUM(closing_stock_value) OVER (PARTITION BY store_id, product_id ORDER BY calday ASC) closing_stock_value,
                    calday date_from,
                    LEAD(calday) OVER (PARTITION BY store_id, product_id ORDER BY calday ASC) date_to
                FROM (
                    SELECT calday, store_id, product_id, closing_stock_quantity, closing_stock_value 
                    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_inventory
                    WHERE calday >=  {START_DATE} AND calday <= {END_DATE} AND calday <= CURRENT_DATE()
                    
                    UNION ALL
                    
                    SELECT calday, store_id, product_id, closing_stock_quantity, closing_stock_value 
                    FROM   {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory  
                    WHERE calday = DATE_SUB( {START_DATE}, 1)
                ) a
            ) i
            ON date_gen >= calday AND (date_gen < date_to OR date_to IS NULL)
        ) a WHERE closing_stock_quantity != 0 OR closing_stock_value != 0 OR rn = 1
    ),

    wcm AS (
        SELECT a.*, IFNULL(b.addr_numbr, c.addr_numbr) addr_numbr
        FROM        main a
        LEFT JOIN   add_rec b ON a.store_id = b.plant
        LEFT JOIN   {catalog_name}.udp_wcm_gold_vcm_dwh.d_franchise_config c ON a.store_id = c.store_id
    ),

    franchise AS (
        SELECT  
            calday - 1 calday,
            a.store_id,
            a.article_id AS product_id,
            a.stock_qty AS closing_stock_quantity,
            IFNULL(a.cost_price, '0') * IFNULL(a.stock_qty, '0') AS closing_stock_value,
            c.addr_numbr
        FROM {catalog_name}.udp_wcm_silver_rop.fms_skustock a
        JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_franchise_config c ON a.store_id = c.store_id
        WHERE DATE_SUB(a.calday, 1) BETWEEN DATE({START_DATE}) AND DATE({END_DATE}) AND a.stock_qty IS NOT NULL
    ), 
    
    total AS (
        SELECT      calday,
                    CASE WHEN c.store_id IS NOT NULL 
                    AND a.calday >= c.cutoff_date 
                    THEN c.store_id ELSE a.store_id END AS store_id,
                    product_id,
                    closing_stock_quantity,
                    closing_stock_value,
                    a.addr_numbr
        FROM        wcm a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_franchise_config c 
        ON a.store_id = c.ref_fc_store
        UNION ALL
        SELECT  calday,
                store_id,
                product_id,
                closing_stock_quantity,
                closing_stock_value,
                addr_numbr
        FROM    franchise
    )
    SELECT      
        FARM_FINGERPRINT(CONCAT(
            IFNULL(CAST(calday AS STRING), ""), 
            IFNULL(CAST(addr_numbr AS STRING), ""), 
            IFNULL(CAST(product_id AS STRING), "")
    )) hash_id,
    calday,
    store_id,
    product_id,
    SUM(closing_stock_quantity) closing_stock_quantity,
    SUM(closing_stock_value) closing_stock_value,
    addr_numbr
    FROM total
    GROUP BY hash_id, calday, store_id, product_id, addr_numbr
    """)

In [0]:
if CALDAY_IN_STR != '':
    # Get the calday list string from temp table
    calday_query = """
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_f_inventory
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]

    # Only run DELETE if the string is non-empt
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory
        WHERE calday IN ({calday_in_str})
        """)
    else:
        print("No valid CALDAYs found to delete.")
        
else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
spark.sql(f"""DROP VIEW IF EXISTS temp_f_inventory""")

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"